In [1]:
import pandas as pd
import numpy as np

In [2]:
dt_2014 = pd.read_csv("datasets/constituency_wise_results_2014.csv")
dt_2019 = pd.read_csv("datasets/constituency_wise_results_2019.csv")

In [3]:
dt_2014.sample(5)

,state,pc_name,candidate,sex,age,category,party,party_symbol,general_votes,postal_votes,total_votes,total_electors
4545,Punjab,Firozpur,SATNAM PAUL KAMBOJ,M,59.0,GEN,AAAP,Broom,113297,115,113412,1522111
8133,Chandigarh,CHANDIGARH,SURINDER BHARDWAJ,M,65.0,GEN,JD(U),Arrow,434,0,434,615214
300,Andhra Pradesh,Srikakulam,REDDY SHANTHI,F,45.0,GEN,YSRCP,Ceiling Fan,426135,2718,428853,1413726
3035,Madhya Pradesh,SAGAR,GOVIND SINGH RAJPUT,M,48.0,GEN,INC,Hand,361811,32,361843,1520322
5474,Tamil Nadu,Dindigul,DHANDAPANI.K,M,42.0,SC,IND,Dish Antenna,863,0,863,1400531


In [4]:
def data_corrector(dataframe):
    dataframe["pc_name"] = np.where((dataframe["state"] == "Himachal Pradesh") & (dataframe["pc_name"] == "Hamirpur"), "Hamirpur (Himachal Pradesh)",
    np.where((dataframe["state"] == "Bihar") & (dataframe["pc_name"] == "Aurangabad"), "Aurangabad (Bihar)",
    np.where((dataframe["state"] == "Bihar") & (dataframe["pc_name"] == "Maharajganj"), "Maharajganj (Bihar)",
    np.where((dataframe["state"] == "Uttar Prades") & (dataframe["pc_name"] == "Hamirpur"), "Hamirpur (Uttar Prades)",
    np.where((dataframe["state"] == "Uttar Prades") & (dataframe["pc_name"] == "Maharajganj"), "Maharajganj (Uttar Prades)",
    np.where((dataframe["state"] == "Maharashtra") & (dataframe["pc_name"] == "Aurangabad"), "Aurangabad (Maharashtra)",dataframe["pc_name"]))))))

In [5]:
data_corrector(dt_2014)
data_corrector(dt_2019)

In [6]:
Telangana_pc_name = ['Adilabad','Peddapalle','Karimnagar','Nizamabad','Zahirabad','Medak','Malkajgiri','Secundrabad','Hyderabad','CHEVELLA','Mahbubnagar','Nagarkurnool','Nalgonda','Bhongir','Warangal','Mahabubabad','Khammam']

In [7]:
dt_2014.loc[dt_2014['pc_name'].isin(Telangana_pc_name), 'state'] = 'Telangana'

In [8]:
canges_for_2014 = {"Dadar & Nagar Haveli":"Dadra And Nagar Haveli","Joynagar":"Jaynagar"}
canges_for_2019 = {"Bikaner (SC)":"Bikaner","CHEVELLA":"CHELVELLA"}

In [9]:
dt_2014["pc_name"] = dt_2014["pc_name"].replace(canges_for_2014)
dt_2019["pc_name"] = dt_2019["pc_name"].replace(canges_for_2019)

# 1

In [10]:
group = dt_2014.groupby("pc_name")

In [11]:
temp_df1 = group["total_votes"].sum().to_frame()

In [12]:
temp_df2 = group["total_electors"].mean().astype(int).to_frame()

In [13]:
df = pd.merge(temp_df1,temp_df2, on = "pc_name")

In [14]:
df["Turnout Ratio"] = round((df["total_votes"] / df["total_electors"])*100,1)

In [15]:
print("top 5 constituencies of 2014 in terms of voter turnout ratio.")
df.sort_values("Turnout Ratio",ascending = False).head(5)

top 5 constituencies of 2014 in terms of voter turnout ratio.


,total_votes,total_electors,Turnout Ratio
pc_name,,,
Dhubri,1369624,1550166,88.4
Nagaland,1038910,1182972,87.8
Tamluk,1337684,1527273,87.6
Bishnupur,1272070,1466921,86.7
Kanthi,1290811,1490409,86.6


In [16]:
print("bottom 5 constituencies of 2014 in terms of voter turnout ratio.")
df.sort_values("Turnout Ratio").head(5)

bottom 5 constituencies of 2014 in terms of voter turnout ratio.


,total_votes,total_electors,Turnout Ratio
pc_name,,,
Srinagar,312212,1207230,25.9
Anantnag,375279,1301143,28.8
Baramulla,465992,1190766,39.1
Kalyan,824196,1922034,42.9
Patna Sahib,882262,1946249,45.3


# 2

In [17]:
group = dt_2019.groupby("pc_name")

In [18]:
temp_df1 = group["total_votes"].sum().to_frame()

In [19]:
temp_df2 = group["total_electors"].mean().astype(int).to_frame()

In [20]:
df = pd.merge(temp_df1,temp_df2, on = "pc_name")

In [21]:
df["Turnout Ratio"] = round((df["total_votes"] / df["total_electors"])*100,1)

In [22]:
print("top 5 constituencies of 2019 in terms of voter turnout ratio.")
df.sort_values("Turnout Ratio",ascending = False).head(5)

top 5 constituencies of 2019 in terms of voter turnout ratio.


,total_votes,total_electors,Turnout Ratio
pc_name,,,
Dhubri,1685005,1858566,90.7
Bishnupur,1420648,1627199,87.3
Barpeta,1458549,1685149,86.6
ARUNACHAL EAST,293781,339788,86.5
Jalpaiguri,1500922,1735464,86.5


In [23]:
print("bottom 5 constituencies of 2019 in terms of voter turnout ratio.")
df.sort_values("Turnout Ratio").head(5)

bottom 5 constituencies of 2019 in terms of voter turnout ratio.


,total_votes,total_electors,Turnout Ratio
pc_name,,,
Anantnag,124896,1397779,8.9
Srinagar,186832,1294671,14.4
Baramulla,455550,1317738,34.6
Hyderabad,877872,1957931,44.8
Kalyan,890313,1965676,45.3


# 3

In [24]:
group = dt_2014.groupby("pc_name")

In [25]:
temp_df1 = group["total_votes"].sum().to_frame()

In [26]:
temp_df2 = group["total_electors"].mean().astype(int).to_frame()

In [27]:
temp_df3 = group["state"].apply(lambda x: x.unique()[0]).to_frame()

In [28]:
df = pd.merge(temp_df1,temp_df2, on = "pc_name")

In [29]:
df = pd.merge(df,temp_df3, on = "pc_name")

In [30]:
df.set_index("state", inplace = True)

In [31]:
df = df.groupby("state").sum()

In [32]:
df["Turnout Ratio"] = round((df["total_votes"] / df["total_electors"])*100,1)

In [33]:
print("top 5 states of 2014 in terms of voter turnout ratio.")
df.sort_values("Turnout Ratio",ascending = False).head(5)

top 5 states of 2014 in terms of voter turnout ratio.


,total_votes,total_electors,Turnout Ratio
state,,,
Nagaland,1038910,1182972,87.8
Lakshadweep,43239,49922,86.6
Tripura,2023829,2388822,84.7
Dadra & Nagar Haveli,165286,196597,84.1
Sikkim,308967,370770,83.3


In [34]:
print("bottom 5 states of 2014 in terms of voter turnout ratio.")
df.sort_values("Turnout Ratio").head(5)

bottom 5 states of 2014 in terms of voter turnout ratio.


,total_votes,total_electors,Turnout Ratio
state,,,
Jammu & Kashmir,3566863,7183129,49.7
Bihar,35885332,63800160,56.2
Uttar Pradesh,81093120,138810557,58.4
Maharashtra,48717419,80798823,60.3
Madhya Pradesh,29639807,48121301,61.6


# 4

In [35]:
group = dt_2019.groupby("pc_name")

In [36]:
temp_df1 = group["total_votes"].sum().to_frame()

In [37]:
temp_df2 = group["total_electors"].mean().astype(int).to_frame()

In [38]:
temp_df3 = group["state"].apply(lambda x: x.unique()[0]).to_frame()

In [39]:
df = pd.merge(temp_df1,temp_df2, on = "pc_name")

In [40]:
df = pd.merge(df,temp_df3, on = "pc_name")

In [41]:
df.set_index("state", inplace = True)

In [42]:
df = df.groupby("state").sum()

In [43]:
df["Turnout Ratio"] = round((df["total_votes"] / df["total_electors"])*100,1)

In [44]:
print("top 5 states of 2019 in terms of voter turnout ratio.")
df.sort_values("Turnout Ratio",ascending = False).head(5)

top 5 states of 2019 in terms of voter turnout ratio.


,total_votes,total_electors,Turnout Ratio
state,,,
Lakshadweep,47009,55189,85.2
Nagaland,1006357,1213777,82.9
Manipur,1617330,1959563,82.5
Tripura,2153172,2614718,82.3
West Bengal,57206976,70001284,81.7


In [45]:
print("bottom 5 states of 2019 in terms of voter turnout ratio.")
df.sort_values("Turnout Ratio").head(5)

bottom 5 states of 2019 in terms of voter turnout ratio.


,total_votes,total_electors,Turnout Ratio
state,,,
Jammu & Kashmir,3552622,7922538,44.8
Bihar,40806661,71216290,57.3
Uttar Pradesh,86481398,146134603,59.2
NCT OF Delhi,8679012,14327649,60.6
Maharashtra,54054245,88676946,61.0


# 5

In [46]:
dt_14 = dt_2014[["pc_name","party","total_votes","total_electors"]].groupby("pc_name").apply(lambda x: x.loc[x["total_votes"].idxmax()]).reset_index(drop=True)
dt_19 = dt_2019[["pc_name","party","total_votes","total_electors"]].groupby("pc_name").apply(lambda x: x.loc[x["total_votes"].idxmax()]).reset_index(drop=True)

In [47]:
df = pd.merge(dt_14,dt_19, on = "pc_name", suffixes=('_14', '_19'))

In [48]:
df = df[df["party_14"] == df["party_19"]]

In [49]:
df["percentage_14"] = round(((df["total_votes_14"]/df["total_electors_14"])*100),2)
df["percentage_19"] = round(((df["total_votes_19"]/df["total_electors_19"])*100),2)

In [50]:
df = df.sort_values("percentage_19",ascending = False)[["pc_name","party_14","percentage_14","percentage_19"]]

In [76]:
print("constituencies which elected the same party for two consecutive elections, rank them by % of votes to that winning party in 2019.")
df.rename(columns={"party_14":"party","pc_name":"constituencies"}).set_index("constituencies").head(15)

constituencies which elected the same party for two consecutive elections, rank them by % of votes to that winning party in 2019.


,party,percentage_14,party_19,percentage_19,diffrence
constituencies,,,,,
Alipurduars,AITC,24.64,BJP,45.55,20.91
Autonomous District,INC,30.35,BJP,47.91,17.56
Puducherry,AINRC,28.38,INC,45.71,17.33
Kanniyakumari,BJP,25.41,INC,41.78,16.37
Chikkballapur,INC,25.61,BJP,41.24,15.63
Kolar,INC,28.06,BJP,43.51,15.45
Sambhal,BJP,21.28,SP,35.98,14.70
Supaul,INC,21.82,JD(U),35.33,13.51
Giridih,BJP,25.93,AJSUP,39.30,13.37


# 6

In [54]:
dt_14 = dt_2014[["pc_name","party","total_votes","total_electors"]].groupby("pc_name").apply(lambda x: x.loc[x["total_votes"].idxmax()]).reset_index(drop=True)
dt_19 = dt_2019[["pc_name","party","total_votes","total_electors"]].groupby("pc_name").apply(lambda x: x.loc[x["total_votes"].idxmax()]).reset_index(drop=True)

In [55]:
df = pd.merge(dt_14,dt_19, on = "pc_name", suffixes=('_14', '_19'))

In [56]:
df = df[df["party_14"] != df["party_19"]]

In [61]:
df["percentage_14"] = round(((df["total_votes_14"]/df["total_electors_14"])*100),2)
df["percentage_19"] = round(((df["total_votes_19"]/df["total_electors_19"])*100),2)
df["diffrence"] = df["percentage_19"] - df["percentage_14"]

In [68]:
df = df.sort_values("diffrence",ascending = False)[["pc_name","party_14","percentage_14","party_19","percentage_19","diffrence"]]

In [75]:
print("top 10 constituencies which voted for different parties in two elections \non the based of difference (2019-2014) in winner vote percentage in two elections.")
df.rename(columns={"pc_name":"constituencies"}).set_index("constituencies").head(10)

top 10 constituencies which voted for different parties in two elections 
on the based of difference (2019-2014) in winner vote percentage in two elections.


,party_14,percentage_14,party_19,percentage_19,diffrence
constituencies,,,,,
Alipurduars,AITC,24.64,BJP,45.55,20.91
Autonomous District,INC,30.35,BJP,47.91,17.56
Puducherry,AINRC,28.38,INC,45.71,17.33
Kanniyakumari,BJP,25.41,INC,41.78,16.37
Chikkballapur,INC,25.61,BJP,41.24,15.63
Kolar,INC,28.06,BJP,43.51,15.45
Sambhal,BJP,21.28,SP,35.98,14.70
Supaul,INC,21.82,JD(U),35.33,13.51
Giridih,BJP,25.93,AJSUP,39.30,13.37
